In [1]:
from src.pispec import PiSpec
from src.trace_utils import TraceParams
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from io import StringIO
import time

# initialize the interface0 v
pispec = PiSpec()
print(pispec.tracecontroller.device)

# alternate the datapoints between two detectors? 
# Thats a tracecontroller things

# the tracecontroller is broken

# pispec.tracecontroller._debug()



waiting for serial... 0.000501129150390625
/dev/ttyACM2


In [2]:
# %timeit TraceParams(meas_led_vis = 6)
# %timeit -r 1 -n 1000 pispec.init_experiment(exp_name="hat")

    
    
%timeit param = TraceParams(meas_led_vis = 6)
param = TraceParams(meas_led_vis = 6)
    
    


2.54 µs ± 16.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [3]:
%timeit pispec.init_experiment(exp_name="timeit")
dest_path = pispec.init_experiment(exp_name="timeit")
    


23.7 µs ± 50.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
%timeit pispec.setup_trace(param)
    


85.6 µs ± 474 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
%timeit pispec.run_trace(timeout_s=0.5)


1.85 s ± 454 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit pispec.conclude_experiment()
  


561 ns ± 3.48 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [20]:

def single_trace(exp_name: str = "single_test"):    
    param = TraceParams(meas_led_vis = 6, pulse_interval= 1000, num_points = 2000)
        
    dest_path = pispec.init_experiment(exp_name=exp_name)
    
    pispec.setup_trace(param)
    
    status = pispec.run_trace(timeout_s=20.0)
    
    pispec.conclude_experiment()
    
single_trace()

tdata = pispec.datahandler.trace_buffers[0]

print(tdata.buffer[-100:])

, 0, 0, 0, 0
584,292584, 0, 0, 0, 0, 0, 0, 0, 0, 0
585,293085, 0, 0, 0, 0, 0, 0, 0, 0, 0
586,2935


In [ ]:
tdata = pispec.datahandler.trace_buffers[0]

print(tdata.buffer[-100:])
f = StringIO(tdata.buffer)

df = pd.read_csv(f, header=None, names=tdata.col_names[0])

print(df.shape)

print(np.min(df["time_us"]), np.max(df["time_us"]))


In [ ]:
print(pispec.set_power_state(0))
print(pispec.set_actinic_intensity(0))
print(pispec.set_actinic_state(0))

# pispec.meas_led_testJ()
# pispec.actinic_test()

### Experiment Test

In [14]:
# pispec = PiSpec()

print("PiSpec connected.")
#experiment variables
exp_name = "p700_test"

print("creating parameters")
trace_params = [TraceParams(num_points=1000,
    pulse_interval = 500,
    meas_led_ir = 0,
    meas_led_vis = x,
    pulse_length = 64,
    sat_pulse_begin = 425,
    sat_pulse_end = 525,
    pulse_mode = 1,
    trigger_delay = 35,
    trace_note = exp_name,
    act_intensity = [0, 140, 0]
) for x in range(6, 8)]

# begin experiment
print("init experiment")
dest_path = pispec.init_experiment(exp_name=exp_name)

print(f"dest_path: {dest_path}")
print("begin trace run")
for i, param in enumerate(trace_params):

    # setup the trace paramaters
    print(f"trace {i}, params: {param.param_string}")
    pispec.setup_trace(param)
    print("run trace")  
    status = pispec.run_trace(timeout_s=2.5)
    print(status)
    pispec.wait(1)
    print("trace complete")

pispec.conclude_experiment()

PiSpec connected.
creating parameters
init experiment
dest_path: /home/pi/projects/pi-spec-cli/export/221006_p700_test
begin trace run
trace 0, params: e35;i500;j0;n1000;p64;r0;s425;t525;v6;w0;x140;y0;z1
run trace
None
trace complete
trace 1, params: e35;i500;j0;n1000;p64;r0;s425;t525;v7;w0;x140;y0;z1
run trace
None
trace complete


'Experiment concluded, trace data is ready for processing.'

In [15]:
tdata = pispec.datahandler.trace_buffers[0]

print(tdata.buffer[-100:])
f = StringIO(tdata.buffer)

df = pd.read_csv(f, header=None, names=tdata.col_names[0])

print(df.shape)

print(np.min(df["time_us"]), np.max(df["time_us"]))


, 0, 0, 0, 0
584,292584, 0, 0, 0, 0, 0, 0, 0, 0, 0
585,293085, 0, 0, 0, 0, 0, 0, 0, 0, 0
586,2935
(587, 11)
0 293085


In [ ]:
import re


def calc_d_abs(df):   
    """ calculates delta A, or delta absorbance
    deltaT = the change in transmission over the course of the trace in voltage terms
    T = mean pre-pulse transmission. use the last 200 pts or so before the saturation pulse.

    d_abs = -(deltaT/T)/2.3
    """
    sat_pulse_begin = re.search('v[0-9]{1,2}', df.loc[0, "param_string"]).group()[1:]

def get_meas_vis_num(df):
    return re.search('v[0-9]{1,2}', df.loc[0, "param_string"]).group()[1:]
    
def get_meas_ir_num(df):
    return re.search('r[0-9]{1,2}', df.loc[0, "param_string"]).group()[1:]


def process_df(df):
    df["val"] = df[['aq_0', 'aq_1', 'aq_2', 'aq_3', 'aq_4']].mean(numeric_only = True, axis=1)
    df["zero_val"] = df[['paq_0', 'paq_1', 'paq_2']].mean(numeric_only = True, axis=1)
    df["V"] = (df["val"]  - df["zero_val"]) * (12 / 65535)
    df["time_ms"] = df["time_us"] / 1000
    df["nm"] = nm_dict[get_meas_vis_num(df)]
    df["d_abs"] = calc_d_abs(df)


    return df

# this should be in config files
nm_dict = {"0":"0", "1":"520", "2":"545", "3":"550", "4":"554", "5":"572", "6":"830", "7":"940", "8":"0"}

dfs = [process_df(df) for df in pispec.get_dataframe_list()]


print(dfs[0].shape)




In [ ]:
# retrieve buffer data from pispec and create our dataframes
# 
import numpy as np




# param_string = dfs[0]["param_string"].split(";")


#proc_dfs = [process_df(df) for df in dfs]

# for df in dfs:
    # # row means of all the data points



In [ ]:


print(np.min(df["pt_num"]), np.max(df["pt_num"]))
df = dfs[0]
df1 = dfs[1]
df["nm"] = 800
df1["nm"] = 900




df["dA"] = 




df = df.append(df1)
df.set_index("time_ms", inplace=True)

print(df.tail())

In [ ]:
# plot it
# plt.ylim(0, 12)
df.groupby("nm")["dAbs"].plot(legend=True)
df.to_csv(f"export/df.csv")

print(df.shape)
print(df.tail())

from 100-190, relative signal
before saturation pulse
delat transmission, change relative to the pre sat pulse transmission

deltaT / T

T = pre saturation pulse

THEN 

dA = (- deltaT/T)/2.3


In [ ]:

# prepulse_mean = df.iloc]
# df["dAbs"] = df["V"] / 

In [ ]:
# import re

# resp = re.findall(pattern="v[0-9]", string=df.param_string[0])
# print(resp)

divide 

In [ ]:
# # row means of all the data points
# df["val"] = df[['aq_0', 'aq_1', 'aq_2', 'aq_3', 'aq_4']].mean(numeric_only = True, axis=1)

# df["V"] = df["val"] / 6535

# plt.plot(df['time_us'], df['V'])
# plt.ylim(0, 5)
# plt.show()

In [ ]:
# plt.plot(df['time_us'], df['V'])
# plt.ylim(0, 10)
# plt.show()

In [ ]:
# (x='time_us', y='val')
# plt.show()

# plt.plot(df['time_us'], df['aq_0'])
# plt.show()

In [ ]:

# pispec.tracecontroller.modify_actinic(intensity=500)
# time.sleep(5)
